<a href="https://colab.research.google.com/github/VinKKAP/Data-Analysis-with-LLM/blob/main/TryTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# Upgrade pip und installiere die benötigten Pakete
%pip install --upgrade pip
%pip install --upgrade torch transformers
import torch

# Überprüfen, ob CUDA verfügbar ist, und den Namen des CUDA-Geräts abrufen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

python(35406) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


python(35407) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [4]:
device

device(type='cpu')

GPU ist verbunden für die berechnung. Aber die Instruction für das Modell scheitert daran das dass jeweilige Modell nicht richtig im Code aus dem Buch funktioniert.

In [1]:
import argparse
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import re
import time

def create_prompt(question):  #1
    """ Generate prompt to translate question into SQL query.

    Args:
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Database:']
    parts += ['create table games(rank int, name text, platform text,']
    parts += ['year int, genre text, publisher text, americasales numeric,']
    parts += ['eusales numeric, japansales numeric, othersales numeric,']
    parts += ['globalsales numeric);']
    parts += ['Translate this question into SQL query:']
    parts += [question]
    return '\n'.join(parts)

def call_llm(prompt):  #2
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    pipe = pipeline("text-generation", model="Qwen/Qwen2-1.5B-Instruct", max_length=900, device=0)
    for nr_retries in range(1, 4):
        try:
            response = pipe(prompt, max_length=150)
            return response[0]['generated_text']
        except Exception as e:
            print(f"Error: {e}")
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query model!')

if __name__ == '__main__':  #3
    parser = argparse.ArgumentParser()
    parser.add_argument('question', type=str, help='A question about games')
    args = parser.parse_args(['how many games are there?'])

    prompt = create_prompt(args.question)
    answer = call_llm(prompt) #4
    query = re.findall(r'```sql\s*(.*?)\s*```', answer, re.DOTALL)
    if query:
        print(f'SQL: {query[0]}')
    else:
        print("No SQL query found in the response.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


KeyboardInterrupt: 